# Metric-based Quantum Architecture Search

## Required imports

In [ ]:
import numpy as np
import scipy

import time
import os

from itertools import combinations, product
from functools import reduce
import copy

import pennylane as qml
from pennylane.templates.layers import RandomLayers
# from pennylane import numpy as np

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import matplotlib.pyplot as plt

from sklearn.datasets import make_moons

from scipy.stats import spearmanr

# import gym

In [48]:
!pip install torch torchquantum

  Using cached torch-1.13.0-cp37-cp37m-manylinux1_x86_64.whl (890.2 MB)
  Using cached torchquantum-0.1.4-py3-none-any.whl (119 kB)
  Using cached nvidia_cuda_nvrtc_cu11-11.7.99-2-py3-none-manylinux1_x86_64.whl (21.0 MB)
  Using cached nvidia_cudnn_cu11-8.5.0.96-2-py3-none-manylinux1_x86_64.whl (557.1 MB)
  Using cached nvidia_cuda_runtime_cu11-11.7.99-py3-none-manylinux1_x86_64.whl (849 kB)
  Using cached nvidia_cublas_cu11-11.10.3.66-py3-none-manylinux1_x86_64.whl (317.1 MB)
  Using cached qiskit-0.32.1-py3-none-any.whl
  Using cached torchpack-0.3.1-py3-none-any.whl (34 kB)
  Using cached pylatexenc-2.10-py3-none-any.whl
  Using cached numpy-1.21.6-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (15.7 MB)
  Using cached torchvision-0.14.0-cp37-cp37m-manylinux1_x86_64.whl (24.3 MB)
  Using cached tqdm-4.64.1-py2.py3-none-any.whl (78 kB)
  Using cached qiskit_aqua-0.9.5-py3-none-any.whl (2.1 MB)
  Using cached qiskit_ignis-0.6.0-py3-none-any.whl (207 kB)
  Using cached qiski

In [3]:
!pip install pennylane pennylane-qiskit qiskit matplotlib

  Using cached PennyLane-0.25.1-py3-none-any.whl (1.0 MB)
  Using cached PennyLane_qiskit-0.24.0-py3-none-any.whl (34 kB)
  Using cached qiskit-0.38.0-py3-none-any.whl
  Using cached matplotlib-3.5.3-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.whl (11.2 MB)
  Using cached networkx-2.6.3-py3-none-any.whl (1.9 MB)
  Using cached appdirs-1.4.4-py2.py3-none-any.whl (9.6 kB)
  Using cached autoray-0.3.2-py3-none-any.whl (36 kB)
  Using cached toml-0.10.2-py2.py3-none-any.whl (16 kB)
  Using cached PennyLane_Lightning-0.25.1-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (13.6 MB)
  Using cached autograd-1.4-py3-none-any.whl (48 kB)
  Using cached semantic_version-2.10.0-py2.py3-none-any.whl (15 kB)
  Using cached retworkx-0.11.0-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl (1.6 MB)
  Using cached ninja-1.10.2.3-py2.py3-none-manylinux_2_5_x86_64.manylinux1_x86_64.whl (108 kB)
  Using cached mthree-1.0.0-cp37-cp37m-manylinux

## Train random circuits

In [ ]:
x_train = np.genfromtxt('./experiment_data/moons_300/x_train.txt')[:, :4]
x_test = np.genfromtxt('./experiment_data/moons_300/x_test.txt')[:, :4]

y_train = np.genfromtxt('./experiment_data/moons_300/y_train.txt')[:, 0]
y_train = 1 - 2 * y_train
y_test = np.genfromtxt('./experiment_data/moons_300/y_test.txt')[:, 0]
y_test = 1 - 2 * y_test

num_params = [2, 4, 6, 8, 10, 12, 14, 16]

num_qubits = 2
num_embeds = 4

dev = qml.device('lightning.qubit', wires=num_qubits)

for curr_num_params in num_params:
    curr_dir = './random/moons/{}_params'.format(curr_num_params)
    
    if not os.path.exists(curr_dir):
        os.mkdir(curr_dir)
        
    for i in range(5, 25):
        curr_circ_dir = curr_dir + '/circ_{}'.format(i + 1)
        
        if not os.path.exists(curr_circ_dir):
            os.mkdir(curr_circ_dir)
        
        ent_prob = np.random.sample()
        cxz = np.random.sample()
        pauli = 0

        circ_gates, gate_params, inputs_bounds, weights_bounds = generate_true_random_gate_circ(num_qubits, num_embeds, curr_num_params, 
                                                                                                        ent_prob=ent_prob, 
                                                                                                        cxz_prob=cxz * ent_prob,
                                                                                                        pauli_prob=pauli * (
                                                                                                            1 - cxz) * ent_prob)
        
        
        np.savetxt(curr_circ_dir + '/gates.txt', circ_gates, fmt="%s")
        np.savetxt(curr_circ_dir + '/gate_params.txt', gate_params, fmt="%s")
        np.savetxt(curr_circ_dir + '/inputs_bounds.txt', inputs_bounds)
        np.savetxt(curr_circ_dir + '/weights_bounds.txt', weights_bounds)
        
        circ = create_gate_circ(dev, circ_gates, gate_params, inputs_bounds,
                                                        weights_bounds, [0], 'exp', 'adjoint') 

        losses_list = []
        accs_list = []
        
        for j in range(5):
            curr_train_dir = curr_circ_dir + '/run_{}'.format(j + 1)

            if os.path.exists(curr_train_dir):
                pass
            else:
                os.mkdir(curr_train_dir)
                        
            info = train_qnn(circ, x_train, y_train, x_test, y_test, [weights_bounds[-1]], 2400, 0.05, 1, mse_loss, verbosity=17300, 
                                                                                            loss_window=50, init_params=None, 
                                                                                            acc_thres=1.1, shuffle=True, print_loss=50)

            val_exps = [circ(x_test[i], info[-1][-1]) for i in range(len(x_test))]
            val_loss = np.array([mse_loss(y_test[k], val_exps[k]) for k in range(len(x_test))]).flatten()

            acc = np.mean(val_loss < 1)

            np.savetxt(curr_train_dir + '/params_{}.txt'.format(j + 1), info[-1])
            np.savetxt(curr_train_dir + '/losses_{}.txt'.format(j + 1), info[0])

            losses_list.append(val_loss)
            accs_list.append(acc)

        np.savetxt(curr_circ_dir + '/accs.txt', accs_list)
        np.savetxt(curr_circ_dir + '/val_losses.txt', losses_list)   

Step 1 | Sliding Loss Window : 0.5090946725113006
Step 51 | Sliding Loss Window : 1.473401100448352
Step 101 | Sliding Loss Window : 1.3324270156342677
Step 151 | Sliding Loss Window : 1.2022767016084788
Step 201 | Sliding Loss Window : 1.0311100251546768
Step 251 | Sliding Loss Window : 1.1623297266142183
Step 301 | Sliding Loss Window : 0.9487044276819863
Step 351 | Sliding Loss Window : 1.066045321344098
Step 401 | Sliding Loss Window : 1.364970575000862
Step 451 | Sliding Loss Window : 1.157791258840586
Step 501 | Sliding Loss Window : 1.2943624942487866
Step 551 | Sliding Loss Window : 0.9865975897335375
Step 601 | Sliding Loss Window : 1.2368556587695239
Step 651 | Sliding Loss Window : 0.9256806265752354
Step 701 | Sliding Loss Window : 1.1422292551604065
Step 751 | Sliding Loss Window : 1.064140061183664
Step 801 | Sliding Loss Window : 0.8961806705631256
Step 851 | Sliding Loss Window : 1.2357098161078417
Step 901 | Sliding Loss Window : 1.3835332308656083
Step 951 | Sliding L

## Train human design circuits

In [ ]:
from datasets import load_dataset
from create_human_design_circs import generate_human_design_circ
from train_circ import mse_loss, mse_vec_loss

import os
import numpy as np

dataset = 'vowel_4'
num_reps = 2

num_qubits = 4
dev = qml.device('lightning.qubit', wires=num_qubits)
meas_qubits = [0, 1]

num_params = [24, 28, 32, 36, 40]
num_embed_layers = 1

num_train_steps = 3000
batch_size = 1
loss_fn = mse_vec_loss

ours_dir = './human_design/amp_basic/vowel_4/'

if not os.path.exists(ours_dir):
    os.mkdir(ours_dir)

x_train, y_train, x_test, y_test = load_dataset(dataset, 'angle', num_reps)
x_train = x_train[:, :16].reshape((len(x_train), 1, 16))
x_test = x_test[:, :16].reshape((len(x_test), 1, 16))
# x_train = x_train.reshape((len(x_train), num_embed_layers, num_qubits))
# x_test = x_test.reshape((len(x_test), num_embed_layers, num_qubits))

dev = qml.device('lightning.qubit', wires=num_qubits)

var_layer_options = [[qml.RY], [qml.RX]] * 10
enc_layer_options = [[None, True], [None, True]] * 2
# enc_layer_options = [['Y'], ['X']] * 2

for curr_num_params in num_params:
    curr_dir = ours_dir + '{}_params'.format(curr_num_params)
    
    if not os.path.exists(curr_dir):
        os.mkdir(curr_dir)
        
    curr_num_layers = curr_num_params // num_qubits
    curr_weights_shape = (curr_num_layers, 1, num_qubits)
        
    for i in range(1):
        curr_circ_dir = curr_dir

        circ = generate_human_design_circ(dev, num_qubits, 'amp', 'basic', num_embed_layers, curr_num_layers, enc_layer_options[:num_embed_layers],
                                          var_layer_options[:curr_num_layers], meas_qubits, 'exp')
        
        losses_list = []
        accs_list = []
        
        for j in range(5):
            curr_train_dir = curr_circ_dir + '/run_{}'.format(j + 1)

            if os.path.exists(curr_train_dir):
                pass
            else:
                os.mkdir(curr_train_dir)
                        
            info = train_qnn(circ, x_train, y_train, x_test, y_test, curr_weights_shape, num_train_steps, 0.05, batch_size, loss_fn, verbosity=17300, 
                                                                                            loss_window=50, init_params=None, 
                                                                                            acc_thres=1.1, shuffle=True, print_loss=50)

            val_exps = np.array([circ(x_test[i], info[-1][-1]) for i in range(len(x_test))])
            val_loss = np.array([loss_fn(y_test[k], val_exps[k]) for k in range(len(x_test))]).flatten()

            acc = np.mean(np.sum(np.multiply(val_exps, y_test) > 0, 1) == 2)

            shape = np.array(info[-1]).shape
            
            np.savetxt(curr_train_dir + '/params_{}.txt'.format(j + 1), np.array(info[-1]).reshape(shape[0], np.product(shape[1:])))
            np.savetxt(curr_train_dir + '/losses_{}.txt'.format(j + 1), info[0])

            losses_list.append(np.mean(val_loss))
            accs_list.append(acc)

        np.savetxt(curr_circ_dir + '/accs.txt', accs_list)
        np.savetxt(curr_circ_dir + '/val_losses.txt', losses_list) 

Step 1 | Sliding Loss Window : 2.5439028191668096
Step 51 | Sliding Loss Window : 1.9477296876766905
Step 101 | Sliding Loss Window : 1.8748005807733967
Step 151 | Sliding Loss Window : 1.6096567003555842
Step 201 | Sliding Loss Window : 1.7038583714828563
Step 251 | Sliding Loss Window : 1.8502116380204001
Step 301 | Sliding Loss Window : 1.759564535266119
Step 351 | Sliding Loss Window : 1.7018402394098384
Step 401 | Sliding Loss Window : 1.740486384893708
Step 451 | Sliding Loss Window : 1.6260157058270996
Step 501 | Sliding Loss Window : 1.5313011582254763
Step 551 | Sliding Loss Window : 1.717031435839244
Step 601 | Sliding Loss Window : 1.7050558766462465
Step 651 | Sliding Loss Window : 1.651840834808314
Step 701 | Sliding Loss Window : 1.7709342965960975
Step 751 | Sliding Loss Window : 1.5608946635368424
Step 801 | Sliding Loss Window : 1.5948981286605999
Step 851 | Sliding Loss Window : 1.8349686543503383
Step 901 | Sliding Loss Window : 1.648990227028024
Step 951 | Sliding L

In [59]:
from importlib import reload

import train_circ

reload(train_circ)

<module 'train_circ' from '/root/train_circ.py'>

In [ ]:
from datasets_nt import load_dataset
from create_human_design_circs import generate_human_design_circ

dataset = 'bank'
num_reps = 2

num_qubits = 4
dev = qml.device('lightning.qubit', wires=num_qubits)
meas_qubits = [0]

ours_dir = './human_design/angle_basic/bank/'

x_train, y_train, x_test, y_test = load_dataset(dataset, 'angle', num_reps)
x_train = x_train.reshape((len(x_train), 2, -1))
x_test = x_test.reshape((len(x_test), 2, -1))

num_params = [4, 8, 12, 16, 20]
params_per_layer = 4

num_embed_layers = 2

dev = qml.device('lightning.qubit', wires=num_qubits)

var_layer_options = [[qml.RX], [qml.RY]] * 4
enc_layer_options = [['X'], ['Y']]

for curr_num_params in num_params:
    curr_dir = ours_dir + '{}_params'.format(curr_num_params)
    
    if not os.path.exists(curr_dir):
        os.mkdir(curr_dir)
        
    curr_num_layers = curr_num_params // params_per_layer
    curr_weights_shape = (curr_num_layers, 1, num_qubits)
        
    for i in range(1):
        curr_circ_dir = curr_dir

        circ = generate_human_design_circ(dev, num_qubits, 'angle', 'basic', num_embed_layers, curr_num_layers, enc_layer_options,
                                          var_layer_options[:curr_num_layers], meas_qubits, 'exp')
        
        losses_list = []
        accs_list = []
        
        for j in range(5):
            curr_train_dir = curr_circ_dir + '/run_{}'.format(j + 1)

            if os.path.exists(curr_train_dir):
                pass
            else:
                os.mkdir(curr_train_dir)
                        
            info = train_qnn(circ, x_train, y_train, x_test, y_test, curr_weights_shape, 5490, 0.05, 1, mse_loss, verbosity=17300, 
                                                                                            loss_window=50, init_params=None, 
                                                                                            acc_thres=1.1, shuffle=True, print_loss=50)

            val_exps = np.array([circ(x_test[i], info[-1][-1]) for i in range(len(x_test))])
            val_loss = np.array([mse_loss(y_test[k], val_exps[k]) for k in range(len(x_test))]).flatten()

            acc = np.mean(val_loss < 1)

            shape = np.array(info[-1]).shape
            
            np.savetxt(curr_train_dir + '/params_{}.txt'.format(j + 1), np.array(info[-1]).reshape(shape[0], np.product(shape[1:])))
            np.savetxt(curr_train_dir + '/losses_{}.txt'.format(j + 1), info[0])

            losses_list.append(val_loss)
            accs_list.append(acc)

        np.savetxt(curr_circ_dir + '/accs.txt', accs_list)
        np.savetxt(curr_circ_dir + '/val_losses.txt', losses_list) 

Step 1 | Sliding Loss Window : 0.037105581875725394
Step 51 | Sliding Loss Window : 1.0729555437737814
Step 101 | Sliding Loss Window : 0.9294064326418
Step 151 | Sliding Loss Window : 1.075771806883421
Step 201 | Sliding Loss Window : 1.0409593843360125
Step 251 | Sliding Loss Window : 1.0625484188881773
Step 301 | Sliding Loss Window : 0.9659282720501179
Step 351 | Sliding Loss Window : 0.9748863884352725
Step 401 | Sliding Loss Window : 0.9292043165557301
Step 451 | Sliding Loss Window : 0.953811557311297
Step 501 | Sliding Loss Window : 0.9613937130068209
Step 551 | Sliding Loss Window : 0.975079007063158
Step 601 | Sliding Loss Window : 1.0014810975702215
Step 651 | Sliding Loss Window : 0.944552523313777
Step 701 | Sliding Loss Window : 0.9736704515176
Step 751 | Sliding Loss Window : 0.8456009321382596
Step 801 | Sliding Loss Window : 0.9534129159241082
Step 851 | Sliding Loss Window : 0.9576173881668996
Step 901 | Sliding Loss Window : 0.901143444811686
Step 951 | Sliding Loss 

In [ ]:
!python supernet/train_search_fmnist.py --warmup_epochs 0 --steps 50000 --n_search 1 --n_qubits 4 --n_experts 1 --n_layers 4 --n_encode_layers 4 --save "16_params" --data "./experiment_data/fmnist_4/" --save_dir "./supernet/fmnist_4"

2022-09-23 05:08:35.568471: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:460] Initializing the SageMaker Profiler.
2022-09-23 05:08:35.568583: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:105] SageMaker Profiler is not enabled. The timeline writer thread will not be started, future recorded events will be dropped.
2022-09-23 05:08:35.598930: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:460] Initializing the SageMaker Profiler.
(8000, 4, 4) (2000, 4, 4)
subnet: [127741, 247366, 256923, 216937], expert_idx: 0
2022-09-23 05:08:48.785614: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN)to use the following CPU instructions in performance-critical operations:  AVX512F
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-09-23 05:08:48.811194: I tensorflow/core/platform/profile_utils/cpu_utils.cc:104] CPU Frequency: 249

## Supernet

In [43]:
def convert_supernet_circ_into_gate_circ(subnet, num_embeds, layer_rots, layer_cnots, num_qubits, angle_embed=False):
    circ_gates = []
    gate_params = []
    weights_bounds = [0]
    inputs_bounds = [0]

    curr_layers = subnet
    num_cnots = len(layer_cnots)
    
    for i in range(1):
        num_qubits = len(layer_rots[0])

        if angle_embed:
            rots = ['ry', 'rx', 'rz']
            
            for j in range(num_embeds):
                circ_gates += [rots[j % 3] for i in range(num_qubits)]
                gate_params += [[i] for i in range(num_qubits)]
                weights_bounds += [0 for i in range(num_qubits)]
                inputs_bounds += [inputs_bounds[-1] + i + 1 for i in range(2 * num_qubits - 1)]            
        else:
            for j in range(num_embeds):
                circ_gates += ['h' for i in range(num_qubits)] + ['ry' for i in range(num_qubits)] + ['cry' for i in range(num_qubits - 1)]
                gate_params += [[i] for i in range(num_qubits)] * 2 + [[i, i + 1] for i in range(num_qubits - 1)]
                weights_bounds += [0 for i in range(3 * num_qubits - 1)]
                inputs_bounds += [inputs_bounds[-1] for i in range(num_qubits)] + [inputs_bounds[-1] + i + 1 for i in range(2 * num_qubits - 1)]

        for j in range(len(curr_layers)):
            circ_gates += layer_rots[curr_layers[j] // num_cnots]
            circ_gates += ['cx' for k in layer_cnots[curr_layers[j] % num_cnots]]

            gate_params += [[k] for k in range(len(layer_rots[curr_layers[j] // num_cnots]))]
            gate_params += layer_cnots[curr_layers[j] % num_cnots]

            weights_bounds += [weights_bounds[-1] + k + 1 for k in range(num_qubits)]
            inputs_bounds += [inputs_bounds[-1] for k in range(num_qubits)]

            weights_bounds += [weights_bounds[-1] for k in range(len(layer_cnots[curr_layers[j] % num_cnots]))]
            inputs_bounds += [inputs_bounds[-1] for k in range(len(layer_cnots[curr_layers[j] % num_cnots]))]
            
    return circ_gates, gate_params, inputs_bounds, weights_bounds

In [44]:
import pennylane as qml
import numpy as np
import os

from datasets_nt import load_dataset
from create_gate_circs import create_gate_circ
from create_noise_models import noisy_dev_from_backend

dataset = 'bank'
main_dir = './supernet/bank/'

x_train, y_train, x_test, y_test = load_dataset(dataset, 'supernet', 2)

# inds = np.random.choice(len(x_test), 250, False)
# x_test = x_test[inds]
# y_test = y_test[inds]

num_qubits = 4
num_cnot_configs = 4096
num_circs = 2500
num_embed_layers = 2

device_name = 'ibmq_lima'
dev = qml.device('lightning.qubit', wires=num_qubits)
# dev = noisy_dev_from_backend(device_name, num_qubits)

param_nums = [12, 20]

for p in param_nums:
    supernet_dir = main_dir + 'search-{}_params_mb'.format(p)

    num_layers = p // num_qubits

    layer_rots = open(supernet_dir + '/rotations.txt').read().split('\n')[:-1]
    layer_rots = [''.join([j for j in i if j.isupper()]) for i in layer_rots]
    layer_rots = [[i[j * 2:j * 2 + 2].lower() for j in range(len(i) // 2)] for i in layer_rots]

    layer_cnots = open(supernet_dir + '/cnots.txt').read().split('\n')[:-1]
    layer_cnots = [''.join([j for j in i[1:] if j not in ['[', ']', ',', ' ']]) for i in layer_cnots]
    layer_cnots = [[[int(i[2 * j]), int(i[2 * j + 1])] for j in range(len(i) // 2)] for i in layer_cnots]
        
    search_space = len(layer_rots) * len(layer_cnots)
        
    accs = []
    losses = []
    circ_layers = []

    params = np.genfromtxt(supernet_dir + '/training_params.txt')[-1].reshape((num_layers, search_space // num_cnot_configs, num_qubits))
        
#     supernet_device_dir = supernet_dir + '/' + device_name
    
#     if not os.path.exists(supernet_device_dir):
#         os.mkdir(supernet_device_dir)
        
    for i in range(num_circs):
        curr_circ_desc = np.random.randint(0, search_space, num_layers)
#         curr_circ_desc = [[272920], [163863], [258091], [172861], [126492]][i]

        curr_params = np.concatenate([params[k, curr_circ_desc[k] // num_cnot_configs] for k in range(num_layers)]).flatten()

        circ_gates, gate_params, inputs_bounds, weights_bounds = convert_supernet_circ_into_gate_circ(curr_circ_desc, num_embed_layers, layer_rots, layer_cnots,
                                                                                                          num_qubits, False) 
        
        circ = create_gate_circ(dev, circ_gates, gate_params, inputs_bounds,
                                                                weights_bounds, [0], 'exp')
        
        val_exp_list = []
        
        for j in range(len(x_test)):
            val_exp_list.append(circ(x_test[j], curr_params))
            
        val_exps = np.array(val_exp_list).flatten()
        val_losses = np.power(val_exps - y_test, 2)
        val_loss = np.mean(val_losses)
        acc = np.mean(val_losses < 1)
    
        print(curr_circ_desc, acc, val_loss, np.sum(val_losses < 1), len(x_test))

        accs.append(acc)
        losses.append(val_loss)
        circ_layers.append(curr_circ_desc)
        
#     np.savetxt(supernet_device_dir + '/searched_circ_layers.txt', np.array(circ_layers))
#     np.savetxt(supernet_device_dir + '/searched_circ_accs.txt', accs)
#     np.savetxt(supernet_device_dir + '/searched_circ_losses.txt', losses)  
    np.savetxt(supernet_dir + '/searched_circ_accs.txt', ['{} {}'.format(circ_layers[i], accs[i]) for i in range(num_circs)], fmt="%s")

[128915 206769  20391] 0.5655737704918032 0.9814923985142966 69 122
[108827 257275 324173] 0.4672131147540984 1.0851017180676144 57 122
[ 27696 184585 164829] 0.4918032786885246 0.954989858057609 60 122
[ 82727 137917  19509] 0.5901639344262295 0.9541414753039731 72 122
[266488 189707  40292] 0.4426229508196721 1.3099861486696343 54 122
[144533  20960 214911] 0.5 0.9826617831897644 61 122
[202444  25365 100409] 0.6639344262295082 0.9392539798220614 81 122
[227721 156698 201605] 0.5655737704918032 1.1882490959035237 69 122
[303104  29190 304429] 0.6229508196721312 1.063005660227047 76 122
[199634   1354  91955] 0.4262295081967213 1.114698322633652 52 122
[ 40275 172131  82037] 0.5737704918032787 0.9790173311242638 70 122
[129046 209854 227316] 0.4098360655737705 1.115363511100136 50 122
[ 54758 105489 118465] 0.4918032786885246 0.9924977058571617 60 122
[ 48361  51066 191961] 0.4672131147540984 1.0972953912246195 57 122
[187550 118861 286540] 0.5163934426229508 1.3928124617851994 63 122

In [22]:
os.path.exists(main_dir + 'search-16_params_mb/search_space.txt')

True

## our metric

In [ ]:
from create_gate_circs import generate_random_gate_circ, generate_true_random_gate_circ, create_gate_circ, create_batched_gate_circ, get_circ_params
from create_human_design_circs import generate_human_design_circ
from metrics import compute_reduced_similarity
from datasets_nt import load_dataset

ideal = np.concatenate((np.ones((16, 16)), np.zeros((16, 16)), np.zeros((16, 16)), np.zeros((16, 16))))
ideal_2 = np.concatenate((np.zeros((16, 16)), np.ones((16, 16)), np.zeros((16, 16)), np.zeros((16, 16))))
ideal = np.concatenate((ideal, ideal_2, ideal_2[::-1, :], ideal[::-1, :]), 1)

# ideal = np.concatenate((np.ones((16, 16)), np.zeros((16, 16))))
# ideal = np.concatenate((ideal, ideal[::-1, :]), 1)

dataset = 'vowel_4'

search_params = [24, 28, 32, 36, 40]

num_qubits = 4
num_embeds = 10
meas_qubits = [0, 1]
samples_per_class = 150

curr_dir = './ours/{}/'.format(dataset)

if not os.path.exists(curr_dir):
    os.mkdir(curr_dir)

x_train, y_train, x_test, y_test = load_dataset(dataset, 'angle', 1)

class_0_sel = np.random.choice(samples_per_class, 16, False)
class_1_sel = np.random.choice(samples_per_class, 16, False) + samples_per_class
class_2_sel = np.random.choice(samples_per_class, 16, False) + (samples_per_class * 2)
class_3_sel = np.random.choice(samples_per_class, 16, False) + (samples_per_class * 3)
sel_inds = np.concatenate((class_0_sel, class_1_sel, class_2_sel, class_3_sel))

# class_0_sel = np.random.choice(samples_per_class, 16, False)
# class_1_sel = np.random.choice(samples_per_class, 16, False) + samples_per_class
# sel_inds = np.concatenate((class_0_sel, class_1_sel))

sel_data = x_train[sel_inds]

num_params = 32
num_data = len(sel_data)

np.savetxt('./ours/{}/sel_data.txt'.format(dataset), sel_data)

dev = qml.device('lightning.qubit', wires=num_qubits)

for p in search_params:
    curr_param_dir = curr_dir + '{}_params'.format(p)
    
    if not os.path.exists(curr_param_dir):
        os.mkdir(curr_param_dir)
    
    d2_min_scores = []
    d2_mean_scores = []
    d2_var_scores = []
    
    d2_t_min_scores = []
    d2_t_mean_scores = []
    d2_t_var_scores = []
    
    mean_mat_scores = []
    mean_t_mat_scores = []

    for i in range(2500):
        curr_circ_dir = curr_param_dir + '/circ_{}'.format(i + 1)

        if not os.path.exists(curr_circ_dir):
            os.mkdir(curr_circ_dir)
        
        ent_prob = np.random.sample()
        cxz = np.random.sample()
        pauli = 0

        params = 2 * np.pi * np.random.sample((num_params, p))
        np.savetxt(curr_circ_dir + '/params.txt', params)
        
        circ_gates, gate_params, inputs_bounds, weights_bounds = generate_true_random_gate_circ(num_qubits, num_embeds, p, 
                                                                                                        ent_prob=ent_prob, 
                                                                                                        cxz_prob=cxz * ent_prob,
                                                                                                        pauli_prob=pauli * (
                                                                                                            1 - cxz) * ent_prob)
        
        np.savetxt(curr_circ_dir + '/gates.txt', circ_gates, fmt="%s")
        np.savetxt(curr_circ_dir + '/gate_params.txt', gate_params, fmt="%s")
        np.savetxt(curr_circ_dir + '/inputs_bounds.txt', inputs_bounds)
        np.savetxt(curr_circ_dir + '/weights_bounds.txt', weights_bounds)

        batched_circ = create_batched_gate_circ(qml.device('lightning.qubit', wires=num_qubits), circ_gates, gate_params, inputs_bounds,
                                            weights_bounds, meas_qubits, 'matrix') 

        circ_d2_scores = []
        circ_d2_t_scores = []
        circ_mean_mat = np.zeros((num_data, num_data))
        circ_t_mean_mat = np.zeros((num_data, num_data))

        for j in range(num_params):
            curr_params = np.concatenate([params[j] for k in range(num_data)]).reshape((num_data, p))
            mat = compute_reduced_similarity(batched_circ, curr_params, sel_data)
            
            t_mat = mat > ((np.sum(mat) - num_data) / (num_data * (num_data - 1)))
        
            diff_mat = mat - ideal
            diff_2d = np.sum(np.multiply(diff_mat, diff_mat))

            diff_t_mat = t_mat - ideal
            diff_2d_t = np.sum(np.multiply(diff_t_mat, diff_t_mat))

            circ_d2_scores.append(diff_2d)
            circ_d2_t_scores.append(diff_2d_t)

            circ_mean_mat += mat / num_params
            circ_t_mean_mat += t_mat / num_params
            
        np.savetxt(curr_circ_dir + '/d2_scores.txt', circ_d2_scores)
        np.savetxt(curr_circ_dir + '/d2_t_scores.txt', circ_d2_t_scores)
        
        d2_min_scores.append(np.min(circ_d2_scores))
        d2_mean_scores.append(np.mean(circ_d2_scores))
        d2_var_scores.append(np.var(circ_d2_scores))
        
        d2_t_min_scores.append(np.min(circ_d2_t_scores))
        d2_t_mean_scores.append(np.mean(circ_d2_t_scores))
        d2_t_var_scores.append(np.var(circ_d2_t_scores))
        
        diff_mean_mat = ideal - circ_mean_mat
        diff_t_mean_mat = ideal - circ_t_mean_mat
        
        mean_mat_scores.append(np.sum(np.multiply(diff_mean_mat, diff_mean_mat)))
        mean_t_mat_scores.append(np.sum(np.multiply(diff_t_mean_mat, diff_t_mean_mat)))
        
        print(i)
        
    np.savetxt(curr_param_dir + '/d2_mean_scores.txt', d2_mean_scores)
    np.savetxt(curr_param_dir + '/d2_min_scores.txt', d2_min_scores)
    np.savetxt(curr_param_dir + '/d2_var_scores.txt', d2_var_scores)
    
    np.savetxt(curr_param_dir + '/d2_t_mean_scores.txt', d2_t_mean_scores)
    np.savetxt(curr_param_dir + '/d2_t_min_scores.txt', d2_t_min_scores)
    np.savetxt(curr_param_dir + '/d2_t_var_scores.txt', d2_t_var_scores)
    
    np.savetxt(curr_param_dir + '/d2_mean_mat_scores.txt', mean_mat_scores)
    np.savetxt(curr_param_dir + '/d2_mean_t_mat_scores.txt', mean_t_mat_scores)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

## Ours + fixed embedding

In [ ]:
from datasets import load_dataset
from create_gate_circs import create_batched_gate_circ
from metrics import compute_reduced_similarity

search_params = [2, 4, 6, 8, 10, 12, 14, 16]

ideal = np.concatenate((np.ones((16, 16)), np.zeros((16, 16))))
ideal = np.concatenate((ideal, ideal[::-1, :]), 1)

num_qubits = 4
num_embeds = 0
meas_qubits = [0]
num_param_runs = 32

x_train, y_train, x_test, y_test = load_dataset('moons_300', 'angle', 2)
main_dir = './ours_fe/moons/angle_2'

class_0_sel = np.random.choice(240, 16, False)
class_1_sel = np.random.choice(240, 16, False) + 240
sel_inds = np.concatenate((class_0_sel, class_1_sel))

sel_data = x_train[sel_inds]
num_data = len(sel_data)

np.savetxt(main_dir + '/sel_data.txt', sel_data)

dev = qml.device('lightning.qubit', wires=num_qubits)

for p in search_params:
    curr_dir = main_dir + '/{}_params'.format(p)
    
    if not os.path.exists(curr_dir):
        os.mkdir(curr_dir)
    
    d2_min_scores = []
    d2_mean_scores = []
    d2_var_scores = []
    
    d2_t_min_scores = []
    d2_t_mean_scores = []
    d2_t_var_scores = []
    
    mean_mat_scores = []
    mean_t_mat_scores = []

    for i in range(2500):
        curr_circ_dir = curr_dir + '/circ_{}'.format(i + 1)

        if not os.path.exists(curr_circ_dir):
            os.mkdir(curr_circ_dir)
        
        ent_prob = np.random.sample()
        cxz = np.random.sample()
        pauli = 0

        params = 2 * np.pi * np.random.sample((num_param_runs, p))
        np.savetxt(curr_circ_dir + '/params.txt', params)
        
        circ_gates, gate_params, inputs_bounds, weights_bounds = generate_true_random_gate_circ(num_qubits, num_embeds, p, 
                                                                                                        ent_prob=ent_prob, 
                                                                                                        cxz_prob=cxz * ent_prob,
                                                                                                        pauli_prob=pauli * (
                                                                                                            1 - cxz) * ent_prob)
        
        circ_gates = ['ry'] * 2 + ['rx'] * 2 + circ_gates
        gate_params = [[0], [1]] * 2 + gate_params
        inputs_bounds = [j for j in range(4)] + [4 for i in range(len(inputs_bounds))]
        weights_bounds = [0 for i in range(4)] + weights_bounds
        
        np.savetxt(curr_circ_dir + '/gates.txt', circ_gates, fmt="%s")
        np.savetxt(curr_circ_dir + '/gate_params.txt', gate_params, fmt="%s")
        np.savetxt(curr_circ_dir + '/inputs_bounds.txt', inputs_bounds)
        np.savetxt(curr_circ_dir + '/weights_bounds.txt', weights_bounds)

        batched_circ = create_batched_gate_circ(qml.device('lightning.qubit', wires=num_qubits), circ_gates, gate_params, inputs_bounds,
                                            weights_bounds, meas_qubits, 'matrix') 

        circ_d2_scores = []
        circ_d2_t_scores = []
        circ_mean_mat = np.zeros((num_data, num_data))
        circ_t_mean_mat = np.zeros((len(sel_data), len(sel_data)))

        for j in range(num_param_runs):
            curr_params = np.concatenate([params[j] for k in range(num_data)]).reshape((num_data, p))
            mat = compute_reduced_similarity(batched_circ, curr_params, sel_data)
            
            t_mat = mat > ((np.sum(mat) - num_data) / (num_data * (num_data - 1)))
        
            diff_mat = mat - ideal
            diff_2d = np.sum(np.multiply(diff_mat, diff_mat))

            diff_t_mat = t_mat - ideal
            diff_2d_t = np.sum(np.multiply(diff_t_mat, diff_t_mat))

            circ_d2_scores.append(diff_2d)
            circ_d2_t_scores.append(diff_2d_t)

            circ_mean_mat += mat / num_param_runs
            circ_t_mean_mat += t_mat / num_param_runs
            
        np.savetxt(curr_circ_dir + '/d2_scores.txt', circ_d2_scores)
        np.savetxt(curr_circ_dir + '/d2_t_scores.txt', circ_d2_t_scores)
        
        d2_min_scores.append(np.min(circ_d2_scores))
        d2_mean_scores.append(np.mean(circ_d2_scores))
        d2_var_scores.append(np.var(circ_d2_scores))
        
        d2_t_min_scores.append(np.min(circ_d2_t_scores))
        d2_t_mean_scores.append(np.mean(circ_d2_t_scores))
        d2_t_var_scores.append(np.var(circ_d2_t_scores))
        
        diff_mean_mat = ideal - circ_mean_mat
        diff_t_mean_mat = ideal - circ_t_mean_mat
        
        mean_mat_scores.append(np.sum(np.multiply(diff_mean_mat, diff_mean_mat)))
        mean_t_mat_scores.append(np.sum(np.multiply(diff_t_mean_mat, diff_t_mean_mat)))
        
        print(i)
        
    np.savetxt(curr_dir + '/d2_mean_scores.txt', d2_mean_scores)
    np.savetxt(curr_dir + '/d2_min_scores.txt', d2_min_scores)
    np.savetxt(curr_dir + '/d2_var_scores.txt', d2_var_scores)
    
    np.savetxt(curr_dir + '/d2_t_mean_scores.txt', d2_t_mean_scores)
    np.savetxt(curr_dir + '/d2_t_min_scores.txt', d2_t_min_scores)
    np.savetxt(curr_dir + '/d2_t_var_scores.txt', d2_t_var_scores)
    
    np.savetxt(curr_dir + '/d2_mean_mat_scores.txt', mean_mat_scores)
    np.savetxt(curr_dir + '/d2_mean_t_mat_scores.txt', mean_t_mat_scores)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

## train our circuits

In [ ]:
from datasets_nt import load_dataset
from create_gate_circs import get_circ_params, create_gate_circ
from train_circ import train_qnn, mse_loss
from create_noise_models import noisy_dev_from_backend

search_nums = [100]
param_nums = [12, 16]

dataset = 'fmnist_4'
num_reps = 1

num_qubits = 4
dev = qml.device('lightning.qubit', wires=num_qubits)
meas_qubits = [0, 1]
mat_size = 4096

device_name = 'ibm_nairobi'
noisy_dev = noisy_dev_from_backend(device_name, num_qubits)

ours_dir = './ours/fmnist_4/'

x_train, y_train, x_test, y_test = load_dataset(dataset, 'angle', num_reps)

for search_num in search_nums:       
    for param_num in param_nums:
        noise_scores = []
        
        for i in range(2500):
            noise_scores.append(np.genfromtxt(ours_dir + '/{}_params/circ_{}/noise_metric/{}/metric_tvd_score.txt'.format(param_num, i + 1, device_name)))
            
        noise_scores = np.array(noise_scores)
        
        mean_t_mat_scores = np.genfromtxt(ours_dir + '{}_params/d2_mean_t_mat_scores.txt'.format(param_num))
        perf_scores = (mat_size - mean_t_mat_scores) / mat_size
        combined_scores = np.multiply(perf_scores, noise_scores)
        
        circ_inds = np.random.permutation(2500)
        
        curr_dir = ours_dir + '/{}_params/search_{}_{}'.format(param_num, search_num, device_name)
        
        if not os.path.exists(curr_dir):
            os.mkdir(curr_dir)

        for j in range(25):
            curr_trial_dir = curr_dir + '/trial_{}'.format(j + 1)

            if not os.path.exists(curr_trial_dir):
                os.mkdir(curr_trial_dir)
        
            sel_inds = circ_inds[range(j * 100, j * 100 + 100)]
            sel_ind = sel_inds[np.argmax(combined_scores[sel_inds])]

            np.savetxt(curr_trial_dir + '/searched_circuit_inds.txt', sel_inds)
            np.savetxt(curr_trial_dir + '/searched_circuit_scores.txt', combined_scores[sel_inds])
            np.savetxt(curr_trial_dir + '/sel_circuit_ind.txt', [sel_ind])
            np.savetxt(curr_trial_dir + '/sel_circuit_score.txt', [combined_scores[sel_ind]])
            
            print(sel_inds, combined_scores[sel_inds])
            print(sel_ind, combined_scores[sel_ind])
            print(noise_scores[sel_ind], perf_scores[sel_ind])

            circ_gates, gate_params, inputs_bounds, weights_bounds = get_circ_params(ours_dir + '{}_params/circ_{}'.format(param_num, sel_ind + 1))

            circ = create_gate_circ(dev, circ_gates, gate_params, inputs_bounds,
                                                                weights_bounds, meas_qubits, 'exp', 'adjoint')
            
            noisy_circ = create_gate_circ(noisy_dev, circ_gates, gate_params, inputs_bounds,
                                                                weights_bounds, meas_qubits, 'exp', None)
        
            losses_list = []
            accs_list = []
            noisy_losses_list = []
            noisy_accs_list = []

            for j in range(5):
                curr_train_dir = curr_trial_dir + '/run_{}'.format(j + 1)

                if os.path.exists(curr_train_dir):
                    pass
                else:
                    os.mkdir(curr_train_dir)


                info = train_qnn(circ, x_train, y_train, x_test, y_test, [weights_bounds[-1]], 16000, 0.05, 1, mse_vec_loss, verbosity=17300, 
                                                                                                loss_window=50, init_params=None, 
                                                                                                acc_thres=1.1, shuffle=True, print_loss=50)

                val_exps = [circ(x_test[i], info[-1][-1]) for i in range(len(x_test))]
                val_loss = np.array([mse_vec_loss(y_test[k], val_exps[k]) for k in range(len(x_test))]).flatten()
                
                noisy_val_exps = [noisy_circ(x_test[i], info[-1][-1]) for i in range(len(x_test))]
                noisy_val_loss = np.array([mse_vec_loss(y_test[k], noisy_val_exps[k]) for k in range(len(x_test))]).flatten()

#                 acc = np.mean(val_loss < 1)
#                 noisy_acc = np.mean(noisy_val_loss < 1)
                acc = np.mean(np.sum(np.multiply(val_exps, y_test) > 0, 1) == 2)
                noisy_acc = np.mean(np.sum(np.multiply(noisy_val_exps, y_test) > 0, 1) == 2)

                np.savetxt(curr_train_dir + '/params_{}.txt'.format(j + 1), info[-1])
                np.savetxt(curr_train_dir + '/losses_{}.txt'.format(j + 1), info[0])             

                losses_list.append(val_loss)
                accs_list.append(acc)
                
                noisy_losses_list.append(noisy_val_loss)
                noisy_accs_list.append(noisy_acc)

            np.savetxt(curr_trial_dir + '/accs.txt', accs_list)
            np.savetxt(curr_trial_dir + '/val_losses.txt', losses_list)          
            np.savetxt(curr_trial_dir + '/noisy_accs.txt', noisy_accs_list)
            np.savetxt(curr_trial_dir + '/noisy_val_losses.txt', noisy_losses_list) 

ibmqfactory.load_account:WARNING:2022-09-28 08:03:19,742: Credentials are already in use. The existing account in the session will be replaced.


[1477    2  409 1016  569  648 1898  886 2067 1476   99 1869  911 2342
 2041  507 1019 2162  387 1685 1062 1107  623 1358 1833 2064  131 1452
 1339  291   50 1038  984 1096  498 1116 1958 1851 1159   22  143 1229
 1282 1244  998 1177 1187 1740  324 1590 1547 1955 1644  358 1669 1133
 2290 1168 2211  162 1344 1625 1374 1745 1664 2183  311 1406 1710  204
 2291 1766 1933 1927 1950 1569 2267 1297 2336 1023  620 1087 2493  546
 2225 1680 2308  471  961 2465 1942  233 1325 1913  833  766 1352  651
 1242 1936] [0.574012   0.56880004 0.72407763 0.65807806 0.66461409 0.70323306
 0.67404135 0.64113455 0.60322894 0.6557359  0.66240215 0.73159681
 0.6749468  0.5338224  0.51635586 0.6401799  0.65965705 0.72191012
 0.72065456 0.74317268 0.70970156 0.73423268 0.6194578  0.68353957
 0.62281999 0.67945605 0.71722323 0.70643493 0.67892187 0.69411576
 0.67575766 0.61859322 0.75030696 0.66082206 0.71593622 0.63788026
 0.57771191 0.74520139 0.48028727 0.58750069 0.45050151 0.71902964
 0.66500414 0.61904102

## metrics for correlation circuitis

In [ ]:
from create_gate_circs import generate_random_gate_circ, generate_true_random_gate_circ, create_gate_circ, create_batched_gate_circ, get_circ_params
from create_human_design_circs import generate_human_design_circ
from metrics import compute_reduced_similarity
from datasets_nt import load_dataset

# ideal = np.concatenate((np.ones((16, 16)), np.zeros((16, 16)), np.zeros((16, 16)), np.zeros((16, 16))))
# ideal_2 = np.concatenate((np.zeros((16, 16)), np.ones((16, 16)), np.zeros((16, 16)), np.zeros((16, 16))))
# ideal = np.concatenate((ideal, ideal_2, ideal_2[::-1, :], ideal[::-1, :]), 1)

ideal = np.concatenate((np.ones((16, 16)), np.zeros((16, 16))))
ideal = np.concatenate((ideal, ideal[::-1, :]), 1)

# ideal = 2 * ideal - 1

dataset = 'fmnist_2'

curr_dir = f'./experiment_data/{dataset}/trained_circuits'
num_qubits = 4
meas_qubits = [0]
num_classes = 2

x_train, y_train, x_test, y_test = load_dataset(dataset, 'angle', 1)
num_samples_per_class = len(x_train) // num_classes

# class_0_sel = np.random.choice(num_samples_per_class, 16, False)
# class_1_sel = np.random.choice(num_samples_per_class, 16, False) + num_samples_per_class
# class_2_sel = np.random.choice(num_samples_per_class, 16, False) + num_samples_per_class * 2
# class_3_sel = np.random.choice(num_samples_per_class, 16, False) + num_samples_per_class * 3
# sel_inds = np.concatenate((class_0_sel, class_1_sel, class_2_sel, class_3_sel))

class_0_sel = np.random.choice(num_samples_per_class, 16, False)
class_1_sel = np.random.choice(num_samples_per_class, 16, False) + num_samples_per_class
sel_inds = np.concatenate((class_0_sel, class_1_sel))

sel_data = x_train[sel_inds]
# sel_data = np.genfromtxt(curr_dir + '/sel_data.txt')

num_params = 32
num_data = len(sel_data)

d2_min_scores = []
d2_mean_scores = []
d2_var_scores = []
    
d2_t_min_scores = []
d2_t_mean_scores = []
d2_t_var_scores = []

mean_mat_scores = []
mean_t_mat_scores = []

np.savetxt(curr_dir + '/sel_data.txt', sel_data)

for i in range(1000):
    circ_dir = curr_dir + '/circ_{}'.format(i + 1)
    
    circ_gates, gate_params, inputs_bounds, weights_bounds = get_circ_params(circ_dir)
    
    batched_circ = create_batched_gate_circ(qml.device('lightning.qubit', wires=num_qubits), circ_gates, gate_params, inputs_bounds,
                                        weights_bounds, meas_qubits, 'matrix') 
    
    params = 2 * np.pi * np.random.sample((num_params, weights_bounds[-1]))
    
    if not os.path.exists(circ_dir + '/fid_mats'):
        os.mkdir(circ_dir + '/fid_mats')
    
    np.savetxt(circ_dir + '/fid_mats/metric_params.txt', params) 
    
    circ_d2_scores = []
    circ_d2_t_scores = []
    circ_mean_mat = np.zeros((num_data, num_data))
    circ_t_mean_mat = np.zeros((num_data, num_data))
    
    for j in range(num_params):
        curr_params = np.concatenate([params[j] for k in range(num_data)]).reshape((num_data, weights_bounds[-1]))
        mat = compute_reduced_similarity(batched_circ, curr_params, sel_data)
        
        t_mat = mat > ((np.sum(mat) - num_data) / (num_data * (num_data - 1)))
        
        diff_mat = mat - ideal
        diff_2d = np.sum(np.multiply(diff_mat, diff_mat))
            
        diff_t_mat = t_mat - ideal
        diff_2d_t = np.sum(np.multiply(diff_t_mat, diff_t_mat))
            
        circ_d2_scores.append(diff_2d)
        circ_d2_t_scores.append(diff_2d_t)
            
        circ_mean_mat += mat / num_params
        circ_t_mean_mat += t_mat / num_params

    np.savetxt(circ_dir + '/fid_mats/d2_scores_2.txt', circ_d2_scores)
    np.savetxt(circ_dir + '/fid_mats/d2_t_scores_2.txt', circ_d2_t_scores)
        
    d2_min_scores.append(np.min(circ_d2_scores))
    d2_mean_scores.append(np.mean(circ_d2_scores))
    d2_var_scores.append(np.var(circ_d2_scores))
        
    d2_t_min_scores.append(np.min(circ_d2_t_scores))
    d2_t_mean_scores.append(np.mean(circ_d2_t_scores))
    d2_t_var_scores.append(np.var(circ_d2_t_scores))
        
    diff_mean_mat = ideal - circ_mean_mat
    diff_t_mean_mat = ideal - circ_t_mean_mat   
    
    mean_mat_scores.append(np.sum(np.multiply(diff_mean_mat, diff_mean_mat)))
    mean_t_mat_scores.append(np.sum(np.multiply(diff_t_mean_mat, diff_t_mean_mat)))
    
    np.savetxt(circ_dir + '/fid_mats/mean_t_mat.txt', circ_t_mean_mat)
    
    print(i)
        
np.savetxt(curr_dir + '/d2_mean_scores_2.txt', d2_mean_scores)
np.savetxt(curr_dir + '/d2_min_scores_2.txt', d2_min_scores)
np.savetxt(curr_dir + '/d2_var_scores_2.txt', d2_var_scores)
    
np.savetxt(curr_dir + '/d2_t_mean_scores_2.txt', d2_t_mean_scores)
np.savetxt(curr_dir + '/d2_t_min_scores_2.txt', d2_t_min_scores)
np.savetxt(curr_dir + '/d2_t_var_scores_2.txt', d2_t_var_scores)
    
np.savetxt(curr_dir + '/d2_mean_mat_scores_kta.txt', mean_mat_scores)
np.savetxt(curr_dir + '/d2_mean_t_mat_scores_kta.txt', mean_t_mat_scores)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

## noise metrics for correlation circuits

In [22]:
from importlib import reload

import metrics

reload(metrics)

<module 'metrics' from '/root/metrics.py'>

In [ ]:
from create_noise_models import get_real_backend_dev, noisy_dev_from_backend
from create_gate_circs import create_batched_gate_circ, get_circ_params
from datasets_nt import load_dataset
from metrics import compute_noise_metric

dataset = 'fmnist_4'

x_train, y_train, _, __ = load_dataset(dataset, 'angle', 1)

curr_dir = './experiment_data/{}/trained_circuits'.format(dataset)

num_qubits = 4
num_trial_params = 128
meas_qubits = [0, 1, 2, 3]

num_cdcs = 128
num_shots = 1024

device_name = 'ibmq_manila'

dev = noisy_dev_from_backend(device_name, num_qubits)
noiseless_dev = qml.device('lightning.qubit', wires=num_qubits)

score_tvds = []
actual_tvds = []

for i in range(1000):
    circ_dir = curr_dir + '/circ_{}'.format(i + 1)
    
    circ_gates, gate_params, inputs_bounds, weights_bounds = get_circ_params(circ_dir)

    noisy_circ = create_batched_gate_circ(dev, circ_gates, gate_params, inputs_bounds,
                                                                weights_bounds, meas_qubits, 'probs') 

    noiseless_circ = create_batched_gate_circ(noiseless_dev, circ_gates, gate_params, inputs_bounds,
                                                                weights_bounds, meas_qubits, 'probs') 
    
    noise_metric_dir = circ_dir + '/noise_metric'
    
    if not os.path.exists(noise_metric_dir):
        os.mkdir(noise_metric_dir)
    
    device_noise_metric_dir = noise_metric_dir + '/{}'.format(device_name)

    if not os.path.exists(device_noise_metric_dir):
        os.mkdir(device_noise_metric_dir)
    
    params = np.random.sample((num_trial_params, weights_bounds[-1])) * 2 * np.pi
    batch_data = x_train[np.random.choice(len(x_train), num_trial_params, False)]
    
    noiseless_res_raw = np.array(noiseless_circ(batch_data, params, shots=num_shots))
    noisy_res_raw = np.array(noisy_circ(batch_data, params, shots=num_shots))

    actual_tvd = 1 - np.mean(np.sum(0.5 * np.abs(noiseless_res_raw - noisy_res_raw), 1))
    
    tvd = compute_noise_metric(circ_gates, gate_params, inputs_bounds, weights_bounds, num_qubits, noiseless_dev, dev, num_cdcs=num_cdcs, num_shots=num_shots)
    
    np.savetxt(device_noise_metric_dir + '/metric_tvd_score.txt', [1 - tvd])
    np.savetxt(device_noise_metric_dir + '/actual_tvd_score.txt', [actual_tvd])
    
    score_tvds.append(1 - tvd)
    actual_tvds.append(actual_tvd)
    
    print(1 - tvd, actual_tvd, i)

# np.savetxt(curr_dir + '/noise_scores.txt', score_tvds)
# np.savetxt(curr_dir + '/noisy_fids.txt', actual_tvds)

ibmqfactory.load_account:WARNING:2022-10-24 03:40:31,645: Credentials are already in use. The existing account in the session will be replaced.


0.883880615234375 0.7763519287109375 0
0.87939453125 0.893341064453125 1
0.8748855590820312 0.88031005859375 2
0.7947845458984375 0.8165359497070312 3
0.8387374877929688 0.8394088745117188 4
0.8800888061523438 0.8688507080078125 5
0.8025054931640625 0.8396987915039062 6
0.86639404296875 0.8534088134765625 7
0.86614990234375 0.86444091796875 8
0.864471435546875 0.87554931640625 9
0.8574981689453125 0.85626220703125 10
0.8842620849609375 0.895721435546875 11
0.8458251953125 0.83465576171875 12
0.8932266235351562 0.9080886840820312 13
0.7443084716796875 0.749267578125 14
0.8756637573242188 0.8894882202148438 15
0.8160324096679688 0.827423095703125 16
0.8331451416015625 0.8523788452148438 17
0.8779449462890625 0.877197265625 18
0.876556396484375 0.8645706176757812 19
0.8564071655273438 0.8552093505859375 20
0.8728561401367188 0.8839340209960938 21
0.8403167724609375 0.8631439208984375 22
0.8688583374023438 0.8786544799804688 23
0.8685989379882812 0.8762664794921875 24
0.8401260375976562 0.

## Train correlation circuits

In [ ]:
from create_noise_models import noisy_dev_from_backend
from datasets_nt import load_dataset
from create_gate_circs import create_gate_circ, get_circ_params
from train_circ import train_qnn, mse_loss

import numpy as np
import os

dataset = 'mnist_2'

x_train, y_train, x_test, y_test = load_dataset(dataset, 'angle', 2)

num_qubits = 4
num_embeds = 32
num_params = 12

device_name = 'ibm_oslo'

# dev = qml.device('lightning.qubit', wires=num_qubits)
dev = noisy_dev_from_backend(device_name, num_qubits)

for i in range(200, 400):
    curr_dir = './experiment_data/{}/trained_circuits/circ_{}'.format(dataset, i + 1)
    circ_gates, gate_params, inputs_bounds, weights_bounds = get_circ_params(curr_dir) 

    circ = create_gate_circ(dev, circ_gates, gate_params, inputs_bounds,
                                                    weights_bounds, [0], 'exp')
    
    
    noiseless_losses = np.genfromtxt(curr_dir + '/val_losses.txt')

    losses_list = []
    accs_list = []
    
    curr_dev_dir = curr_dir + '/' + device_name

#     if not os.path.exists(curr_dev_dir + '/accs_inference_only.txt'):
    if True:
        if not os.path.exists(curr_dev_dir):
            os.mkdir(curr_dev_dir)

        for j in range(5):
            curr_train_dir = curr_dir + '/run_{}'.format(j + 1)
            curr_params = np.genfromtxt(curr_train_dir + '/params_{}.txt'.format(j + 1))[-1]

            val_exps = [circ(x_test[i], curr_params) for i in range(len(x_test))]
            val_loss = np.array([mse_loss(y_test[k], val_exps[k]) for k in range(len(x_test))]).flatten()

#             acc = np.mean(np.sum(np.multiply(val_exps, y_test) > 0, 1) == 2)
            acc = np.mean(val_loss < 1)

            losses_list.append(val_loss)
            accs_list.append(acc)

        print(np.mean(noiseless_losses), np.mean(losses_list), i + 1)

        np.save(curr_dev_dir + '/val_losses_inference_only.npy', losses_list)
        np.savetxt(curr_dev_dir + '/accs_inference_only.txt', accs_list)
    else:
        print(i)

In [ ]:
from datasets_nt import load_dataset
from create_gate_circs import create_gate_circ, get_circ_params
from train_circ import train_qnn, mse_vec_loss

import pennylane as qml
import numpy as np
import os

dataset = 'mnist_4'

x_train, y_train, x_test, y_test = load_dataset(dataset, 'angle', 1)

print(x_train.shape)

num_qubits = 4
num_embeds = 16
num_params = 32

for i in range(440, 550):
    curr_dir = './experiment_data/mnist_4/trained_circuits/circ_{}'.format(i + 1)
    circ_gates, gate_params, inputs_bounds, weights_bounds = get_circ_params(curr_dir) 

    circ = create_gate_circ(qml.device('lightning.qubit', wires=num_qubits), circ_gates, gate_params, inputs_bounds,
                                                    weights_bounds, [0, 1], 'exp')    

    losses_list = []
    accs_list = []
    
    for j in range(1):
        curr_train_dir = curr_dir + '/run_{}'.format(j + 1)

        if os.path.exists(curr_train_dir):
            pass
        else:
            os.mkdir(curr_train_dir)
    
        
        info = train_qnn(circ, x_train, y_train, x_test, y_test, [num_params], 16000, 0.05, 1, mse_vec_loss, verbosity=127300, 
                                                                                        loss_window=50, init_params=None, 
                                                                                        acc_thres=1.1, shuffle=True, print_loss=50)
        
        val_exps = [circ(x_test[i], info[-1][-1]) for i in range(len(x_test))]
        val_loss = np.array([mse_vec_loss(y_test[k], val_exps[k]) for k in range(len(x_test))]).flatten()

        acc = np.mean(np.sum(np.multiply(val_exps, y_test) > 0, 1) == 2)
        
        np.savetxt(curr_train_dir + '/params_{}.txt'.format(j + 1), info[-1])
        np.savetxt(curr_train_dir + '/losses_{}.txt'.format(j + 1), info[0])
        
        losses_list.append(val_loss)
        accs_list.append(acc)
        
    np.savetxt(curr_dir + '/val_losses.txt', losses_list)
    np.savetxt(curr_dir + '/accs.txt', accs_list)

(8000, 16)
Step 1 | Sliding Loss Window : 1.2110797939095763
Step 51 | Sliding Loss Window : 1.340711482895409
Step 101 | Sliding Loss Window : 1.307502774269389
Step 151 | Sliding Loss Window : 1.1597822832667064
Step 201 | Sliding Loss Window : 1.103035206045489
Step 251 | Sliding Loss Window : 0.8153710995017763
Step 301 | Sliding Loss Window : 1.2093201889568446
Step 351 | Sliding Loss Window : 0.9635830262704956
Step 401 | Sliding Loss Window : 1.302424145940006
Step 451 | Sliding Loss Window : 1.1374469079249128
Step 501 | Sliding Loss Window : 1.115714518532166
Step 551 | Sliding Loss Window : 1.0055838371674446
Step 601 | Sliding Loss Window : 1.0779999258713346
Step 651 | Sliding Loss Window : 0.803841871202422
Step 701 | Sliding Loss Window : 1.034581798011169
Step 751 | Sliding Loss Window : 0.9819847958009755
Step 801 | Sliding Loss Window : 1.2653328662260714
Step 851 | Sliding Loss Window : 1.1690008915166055
Step 901 | Sliding Loss Window : 0.8620045200098485
Step 951 | 

In [2]:
from importlib import reload

import datasets
import create_gate_circs
import train_circ

reload(datasets)
reload(create_gate_circs)
reload(train_circ)

/usr/local/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


<module 'train_circ' from '/root/train_circ.py'>

In [ ]:
from datasets import TorchDataset
from create_gate_circs import get_circ_params, TQCirc, generate_random_gate_circ
from train_circ import train_tq_model

import pickle as pkl
import torch
import os
import numpy as np

num_qubits = 10

device = torch.device('cpu')

dataset = 'mnist_10'

train_data = TorchDataset(dataset, 'angle', 1, True)
test_data = TorchDataset(dataset, 'angle', 1, False)

train_data_loader = torch.utils.data.DataLoader(train_data, batch_size=1, sampler=torch.utils.data.RandomSampler(train_data))
test_data_loader = torch.utils.data.DataLoader(train_data, batch_size=1, sampler=torch.utils.data.RandomSampler(train_data))

val_test_data_loader = torch.utils.data.DataLoader(test_data, batch_size=100, sampler=torch.utils.data.SequentialSampler(test_data))

loss = torch.nn.functional.nll_loss
val_loss = torch.nn.NLLLoss(reduction='none')

for i in range(166, 200):
    curr_dir = './experiment_data/{}/trained_circuits/circ_{}'.format(dataset, i + 1)

    circ_gates, gate_params, inputs_bounds, weights_bounds = get_circ_params(curr_dir) 

    model = TQCirc(circ_gates, gate_params, inputs_bounds, weights_bounds, num_qubits, True).to(device)
    opt = torch.optim.SGD(model.parameters(), lr=0.05)
    
    losses_list = []
    accs_list = []
    
    for j in range(1):
        curr_train_dir = curr_dir + '/run_{}'.format(j + 1)

        if os.path.exists(curr_train_dir):
            pass
        else:
            os.mkdir(curr_train_dir)
    
        accs_history = train_tq_model(model, opt, loss, train_data_loader, test_data_loader, 120000, 1000, 1000)
        
#         curr_val_losses = []
#         curr_acc = 0
        
#         for k in range(100):
#             val_x, val_y = next(iter(val_test_data_loader))

#             val_preds = model(val_x)
#             curr_val_losses.append(val_loss(val_preds, val_y.to(torch.long)).detach().numpy().flatten())
        
#             val_class = torch.argmax(val_preds.detach(), 1).detach().numpy().flatten()

#             curr_acc += np.mean(val_class == val_y.detach().numpy().flatten())
        
#         np.save(curr_train_dir + '/accs_history_{}.npy'.format(j + 1), accs_history)
        
#         losses_list.append(curr_val_losses)
#         accs_list.append(curr_acc / 100)
        
#         print(curr_acc)
        
    torch.save(model.state_dict(), curr_dir + '/model.pt')
#     np.save(curr_dir + '/val_losses.npy', losses_list)
#     np.savetxt(curr_dir + '/accs.txt', accs_list)

Step 1 | Accuracy: 0.0
Step 1001 | Accuracy: 0.0
Step 2001 | Accuracy: 1.0
Step 3001 | Accuracy: 0.0
Step 4001 | Accuracy: 1.0
Step 5001 | Accuracy: 1.0
Step 6001 | Accuracy: 1.0
Step 7001 | Accuracy: 1.0
Step 8001 | Accuracy: 0.0
Step 9001 | Accuracy: 1.0
Step 10001 | Accuracy: 0.0
Step 11001 | Accuracy: 0.0
Step 12001 | Accuracy: 0.0
Step 13001 | Accuracy: 0.0
Step 14001 | Accuracy: 1.0
Step 15001 | Accuracy: 1.0
Step 16001 | Accuracy: 1.0
Step 17001 | Accuracy: 0.0
Step 18001 | Accuracy: 0.0
Step 19001 | Accuracy: 0.0
Step 20001 | Accuracy: 1.0
Step 21001 | Accuracy: 1.0
Step 22001 | Accuracy: 1.0
Step 23001 | Accuracy: 0.0
Step 24001 | Accuracy: 0.0
Step 25001 | Accuracy: 1.0
Step 26001 | Accuracy: 1.0
Step 27001 | Accuracy: 0.0
Step 28001 | Accuracy: 1.0
Step 29001 | Accuracy: 0.0
Step 30001 | Accuracy: 0.0
Step 31001 | Accuracy: 1.0
Step 32001 | Accuracy: 1.0
Step 33001 | Accuracy: 0.0
Step 34001 | Accuracy: 0.0
Step 35001 | Accuracy: 0.0
Step 36001 | Accuracy: 1.0
Step 37001 | A

## Train circuits using our method

In [ ]:
from datasets import load_dataset
from create_gate_circs import get_circ_params, create_gate_circ
from train_circ import train_qnn, mse_loss

search_nums = [100]
param_nums = [2, 4, 6, 8, 10, 12, 14, 16]

dataset = 'moons_300'
num_reps = 2

num_qubits = 2
dev = qml.device('lightning.qubit', wires=2)
meas_qubits = [0]

ours_dir = './ours/moons/'

x_train, y_train, x_test, y_test = load_dataset(dataset, 'angle', num_reps)

for search_num in search_nums:       
    for param_num in param_nums:
        curr_dir = ours_dir + '/{}_params/search_{}'.format(param_num, search_num)
        
        if not os.path.exists(curr_dir):
            os.mkdir(curr_dir)
    
        mean_t_mat_scores = np.genfromtxt(ours_dir + '{}_params/d2_mean_t_mat_scores.txt'.format(param_num))

        for j in range(25):
            curr_trial_dir = curr_dir + '/trial_{}'.format(j + 1)

            if not os.path.exists(curr_trial_dir):
                os.mkdir(curr_trial_dir)
        
            sel_inds = range(j * 100, j * 100 + 100)
            sel_ind = sel_inds[np.argmin(mean_t_mat_scores[sel_inds])]

            np.savetxt(curr_trial_dir + '/searched_circuit_inds.txt', sel_inds)
            np.savetxt(curr_trial_dir + '/searched_circuit_scores.txt', mean_t_mat_scores[sel_inds])
            np.savetxt(curr_trial_dir + '/sel_circuit_ind.txt', [sel_ind])

            circ_gates, gate_params, inputs_bounds, weights_bounds = get_circ_params(ours_dir + '{}_params/circ_{}'.format(param_num, sel_ind + 1))

            circ = create_gate_circ(dev, circ_gates, gate_params, inputs_bounds,
                                                                weights_bounds, meas_qubits, 'exp', 'adjoint')
        
            losses_list = []
            accs_list = []

            for j in range(5):
                curr_train_dir = curr_trial_dir + '/run_{}'.format(j + 1)

                if os.path.exists(curr_train_dir):
                    pass
                else:
                    os.mkdir(curr_train_dir)


                info = train_qnn(circ, x_train, y_train, x_test, y_test, [weights_bounds[-1]], 2400, 0.05, 1, mse_loss, verbosity=17300, 
                                                                                                loss_window=50, init_params=None, 
                                                                                                acc_thres=1.1, shuffle=True, print_loss=50)

                val_exps = [circ(x_test[i], info[-1][-1]) for i in range(len(x_test))]
                val_loss = np.array([mse_loss(y_test[k], val_exps[k]) for k in range(len(x_test))]).flatten()

                acc = np.mean(val_loss < 1)

                np.savetxt(curr_train_dir + '/params_{}.txt'.format(j + 1), info[-1])
                np.savetxt(curr_train_dir + '/losses_{}.txt'.format(j + 1), info[0])

                losses_list.append(val_loss)
                accs_list.append(acc)

            np.savetxt(curr_trial_dir + '/accs.txt', accs_list)
            np.savetxt(curr_trial_dir + '/val_losses.txt', losses_list)          

Step 1 | Sliding Loss Window : 0.4103990086853829
Step 51 | Sliding Loss Window : 1.3541394430095945
Step 101 | Sliding Loss Window : 1.343907243530291
Step 151 | Sliding Loss Window : 0.7193665817438675
Step 201 | Sliding Loss Window : 0.4955324082003915
Step 251 | Sliding Loss Window : 0.6483260649913112
Step 301 | Sliding Loss Window : 0.6773127876674616
Step 351 | Sliding Loss Window : 0.6631000536592648
Step 401 | Sliding Loss Window : 0.705941994544655
Step 451 | Sliding Loss Window : 0.6444729666597714
Step 501 | Sliding Loss Window : 0.636277421990264
Step 551 | Sliding Loss Window : 0.6576109658938591
Step 601 | Sliding Loss Window : 0.6406746157315556
Step 651 | Sliding Loss Window : 0.6325582288109954
Step 701 | Sliding Loss Window : 0.515561780016473
Step 751 | Sliding Loss Window : 0.6428548065698694
Step 801 | Sliding Loss Window : 0.6611974446181302
Step 851 | Sliding Loss Window : 0.7118821631951319
Step 901 | Sliding Loss Window : 0.670273272709862
Step 951 | Sliding L

KeyboardInterrupt: 